In [1]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import json
import fileinput
import numpy as np
from pprint import pprint

2022-05-26 19:13:37.641 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")
olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)

#First view of given Dataset
print("Datensatzlänge: " + str(olafScholzTwitter.info()))
print(olafScholzTwitter.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   http_status   65 non-null     int64 
 1   account_name  65 non-null     object
 2   account_data  65 non-null     object
 3   params        65 non-null     object
 4   response      65 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
Datensatzlänge: None
   http_status account_name  \
0          200   OlafScholz   
1          200   OlafScholz   
2          200   OlafScholz   
3          200   OlafScholz   
4          200   OlafScholz   

                                        account_data  \
0  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
1  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
2  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
3  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
4  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   

         

In [3]:
#Analyzin Dictionarys
print("Response Metadaten: " + str(olafScholzTwitter["response"][0].keys()))
ResponseKeys = olafScholzTwitter["response"][0].keys()
print(str(ResponseKeys)+"\n")
for Key in ResponseKeys:
    print("Type of of Response Key {}: ".format(Key) + str(type(olafScholzTwitter["response"][0][Key])))
    #print("{}-Metadaten: ".format(Key) + str(olafScholzTwitter["response"][0][Key]))

Response Metadaten: dict_keys(['data', 'includes', 'meta'])
dict_keys(['data', 'includes', 'meta'])

Type of of Response Key data: <class 'list'>
Type of of Response Key includes: <class 'dict'>
Type of of Response Key meta: <class 'dict'>


In [4]:
#print(olafScholzTwitter["response"][0].len)
#print("---")
#Starting to investigate Dataset
#print(olafScholzTwitter.describe())
#print(type(olafScholzTwitter))
#print(olafScholzTwitter.keys())
#for key in olafScholzTwitter:
#    print("{} : {}".format(key, olafScholzTwitter[key]))
#    print("{}".format(key) + str(type(olafScholzTwitter[key])))
    

In [5]:
#Block for testing Dataset and figuring out Key parameters, comment out before finaly running to get propper result
"""
for line in olafScholzJsonLines:
    #print(line)
    keyResponse = line["response"]
    print("Response-Type:" + str(type(keyResponse)))
    print("Response-Keys:" + str(keyResponse.keys()))
    
    for key in keyResponse:
        print(type(keyResponse[key]))
        #pprint(keyResponse[key])
        #print("{}-Metadaten: ".format(key) + str(keyResponse[key].keys()))
    print("Includes-Metadaten: " + str(keyResponse["includes"].keys())+ "\n")
 
    print("Includes-Keys:" + str(keyResponse["includes"]["users"][0].keys())+"\n")
    #pprint("Example Includes:" +"\n" + str(keyResponse["includes"]["users"])+"\n")
        
    print("Meta-Metadaten: " + str(keyResponse["meta"].keys())+ "\n")
    data = keyResponse["data"]
    print("Type-Data: " + str(type(data))+ "\n")
    print(str(data[0]) + "\n")
    print("Type-Data[0]: " + str(type(data[0])) + "\n")
    print("data[0].keys(): " + str(data[0].keys()) + "\n")
    #print(dir(data[0].keys()))
    print("len(data[0].keys()): " + str(len(data[0].keys())) + "\n")
    break
"""

'\nfor line in olafScholzJsonLines:\n    #print(line)\n    keyResponse = line["response"]\n    print("Response-Type:" + str(type(keyResponse)))\n    print("Response-Keys:" + str(keyResponse.keys()))\n    \n    for key in keyResponse:\n        print(type(keyResponse[key]))\n        #pprint(keyResponse[key])\n        #print("{}-Metadaten: ".format(key) + str(keyResponse[key].keys()))\n    print("Includes-Metadaten: " + str(keyResponse["includes"].keys())+ "\n")\n \n    print("Includes-Keys:" + str(keyResponse["includes"]["users"][0].keys())+"\n")\n    #pprint("Example Includes:" +"\n" + str(keyResponse["includes"]["users"])+"\n")\n        \n    print("Meta-Metadaten: " + str(keyResponse["meta"].keys())+ "\n")\n    data = keyResponse["data"]\n    print("Type-Data: " + str(type(data))+ "\n")\n    print(str(data[0]) + "\n")\n    print("Type-Data[0]: " + str(type(data[0])) + "\n")\n    print("data[0].keys(): " + str(data[0].keys()) + "\n")\n    #print(dir(data[0].keys()))\n    print("len(dat

In [6]:
# Reply Dataframe
# Importing Data into final Dataframe for ML
replyDataFrame = pd.DataFrame(columns = ["target", "ids", "date", "flag", "user", "text"])
print(replyDataFrame.head())
#dir(replyDataFrame)
iterator = 0 #Debugging Counter
for line in olafScholzJsonLines:
    keyResponse = line["response"]
    data = keyResponse["data"]
    #print("Iterator {} has length: ".format(iterator) + str(len(data)))
    #pprint(data)
    #["target", "ids", "date", "flag", "user", "text"]
    newDataRow = [None, None, None, None, None, None]
    #print(line.keys())
    userName = line["account_name"]
    #print("User Name: " + str(line["account_name"])+"\n")
    newDataRow[4] = userName
    keyResponse = line["response"]
    data = keyResponse["data"]


    for tweet in data:
        #print("Tweet-ID: " + tweet["id"])
        #print(str(tweet) + "\n")
        tweetTarget = None
        tweetID = tweet["id"]
        authorID = tweet["author_id"]
        tweetDate = tweet["created_at"][0:10]
        tweetFlag = "NO_QUERY"
        tweetText = tweet["text"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = userName
        #print(type(tweetAuthor))
        
        newDataRow[0] = tweetTarget
        newDataRow[1] = tweetID
        newDataRow[2] = tweetDate
        newDataRow[3] = tweetFlag
        #newDataRow[4] = tweetAuthor
        newDataRow[5] = tweetText
        #pprint(newDataRow)
        replyDataFrame.loc[len(replyDataFrame)] = newDataRow
        #tempDataFrame = pd.DataFrame(newDataRow, columns["target", "ids", "date", "flag", "user", "text"])
        #replyDataFrame = replyDataFrame.append(tempDataFame, ignore_index = True)
        #replyDataFrame = replyDataFrame.append(pd.DataFrame(newDataRow, columns = ["target", "ids", "date", "flag", "user", "text"]),ignore_index = True)
        #break
    #print("{} Finale Dataframe \n".format(iterator) + str(replyDataFrame.head()))
    iterator += 1 


Empty DataFrame
Columns: [target, ids, date, flag, user, text]
Index: []


In [7]:
#print(dir(replyDataFrame))
print(len(replyDataFrame))
print(replyDataFrame.shape)
print(replyDataFrame.head())

6419
(6419, 6)
  target                  ids        date      flag        user  \
0   None  1515588838562971650  2022-04-17  NO_QUERY  OlafScholz   
1   None  1514644785818058757  2022-04-14  NO_QUERY  OlafScholz   
2   None  1514291545742184451  2022-04-13  NO_QUERY  OlafScholz   
3   None  1514256817613905929  2022-04-13  NO_QUERY  OlafScholz   
4   None  1513955069867274245  2022-04-12  NO_QUERY  OlafScholz   

                                                text  
0  RT @Bundeskanzler: Ich wünsche Ihnen und Ihren...  
1  RT @Bundeskanzler: Meine Solidarität gilt @Kar...  
2  RT @Bundeskanzler: Sie haben Hilfe organisiert...  
3  RT @Bundeskanzler: Verantwortung übernehmen he...  
4  RT @Bundeskanzler: Die Sanktionen gegen #Russl...  


In [8]:
#print(replyDataFrame.query("date < '2022-01-01'").head())
#print(replyDataFrame.query("date < '2022-01-01'").count())
#print(replyDataFrame.query("text"[0:18] == "RT @Bundeskanzler:").head())
print("----")
#mask = replyDataFrame["text"].str.contains("RT @Bundeskanzler:").head()
#print(replyDataFrame[mask].head())
#print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].head())
#print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].count())
print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].sort_values("date",ascending = "true").head())
# -> Proven that first Retweet was after 2022-02-14

----
   target                  ids        date      flag        user  \
53   None  1493195813245313024  2022-02-14  NO_QUERY  OlafScholz   
52   None  1493599012691365903  2022-02-15  NO_QUERY  OlafScholz   
51   None  1493602211829587986  2022-02-15  NO_QUERY  OlafScholz   
50   None  1494003008283287560  2022-02-16  NO_QUERY  OlafScholz   
49   None  1494245849085493251  2022-02-17  NO_QUERY  OlafScholz   

                                                 text  
53  RT @Bundeskanzler: Auf dem Weg in die #Ukraine...  
52  RT @Bundeskanzler: Der Frieden und die Sicherh...  
51  RT @Bundeskanzler: Ich habe Präsident #Putin i...  
50  RT @Bundeskanzler: Die #Corona-Regeln zeigen d...  
49  RT @Bundeskanzler: Gestern Abend habe ich mit ...  


In [9]:
#testing
print(replyDataFrame["text"].str.contains("RT @Bundeskanzler").head())
print(replyDataFrame["text"].str.contains("RT @Bundeskanzler").count())

0    True
1    True
2    True
3    True
4    True
Name: text, dtype: bool
6419


In [10]:
#cleansedTweetDF = replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:") or replyDataFrame["text"].str.contains("!RT @") ]
print("Size of dataframe :\n"+ str(replyDataFrame.count())+"\n----")
maskBundesKanz = replyDataFrame["text"].str.contains("RT @Bundeskanzler:")
maskNoRT = replyDataFrame["text"].str.contains("RT @")==False
print("Size of dataframe with RT @Bundeskanzler:\n" + str(replyDataFrame[maskBundesKanz].count())+"\n----")
print("Size of dataframe without RT @:\n" + str(replyDataFrame[maskNoRT].count())+"\n----")
#cleansedTweetDF contains all Tweets posted directly by Olaf before or after being Kanzler
cleansedTweetDF = replyDataFrame[maskBundesKanz | maskNoRT]
print("Size of cleansedTweetDF @:\n" + str(cleansedTweetDF.count())+"\n----")
print(cleansedTweetDF.head())
#cleansedTweetDF has "RT @Bundeskanzler:" still in the tweet. To not have any issues with ML we removed that part of the Tweet in the following parts

Size of dataframe :
target       0
ids       6419
date      6419
flag      6419
user      6419
text      6419
dtype: int64
----
Size of dataframe with RT @Bundeskanzler:
target     0
ids       48
date      48
flag      48
user      48
text      48
dtype: int64
----
Size of dataframe without RT @:
target       0
ids       4071
date      4071
flag      4071
user      4071
text      4071
dtype: int64
----
Size of cleansedTweetDF @:
target       0
ids       4119
date      4119
flag      4119
user      4119
text      4119
dtype: int64
----


In [28]:
# Removing the Text "RT @Bundeskanzler:" from the text
print("ReplyDataframe: \n" + str(replyDataFrame[maskBundesKanz].count()))
cleansedAtBundKanzDF = replyDataFrame[maskBundesKanz].copy()
#print(str(cleansedAtBundKanzDF["text"]))
cleansedAtBundKanzDF.text = cleansedAtBundKanzDF.text.str.strip("RT @Bundeskanzler: ")
print(cleansedAtBundKanzDF.head())

ReplyDataframe: 
target     0
ids       48
date      48
flag      48
user      48
text      48
dtype: int64
  target                  ids        date      flag        user  \
0   None  1515588838562971650  2022-04-17  NO_QUERY  OlafScholz   
1   None  1514644785818058757  2022-04-14  NO_QUERY  OlafScholz   
2   None  1514291545742184451  2022-04-13  NO_QUERY  OlafScholz   
3   None  1514256817613905929  2022-04-13  NO_QUERY  OlafScholz   
4   None  1513955069867274245  2022-04-12  NO_QUERY  OlafScholz   

                                                text  
0  Ich wünsche Ihnen und Ihren Familien frohe #Os...  
1  Meine Solidarität gilt @Karl_Lauterbach: Jeder...  
2  Sie haben Hilfe organisiert, Kräfte gebündelt ...  
3  Verantwortung übernehmen heißt: Deutschland li...  
4  Die Sanktionen gegen #Russland sind notwendig ...  


In [40]:
#appending the sliced versions to the Dataframe and resulting in the expected 4119 Tweets
cleansedTweetDFShort = replyDataFrame[maskNoRT].append(cleansedAtBundKanzDF)
print(cleansedTweetDFShort.count())
print(cleansedTweetDFShort.sort_values("date",ascending = False).head())





target       0
ids       4119
date      4119
flag      4119
user      4119
text      4119
dtype: int64
  target                  ids        date      flag        user  \
0   None  1515588838562971650  2022-04-17  NO_QUERY  OlafScholz   
1   None  1514644785818058757  2022-04-14  NO_QUERY  OlafScholz   
2   None  1514291545742184451  2022-04-13  NO_QUERY  OlafScholz   
3   None  1514256817613905929  2022-04-13  NO_QUERY  OlafScholz   
4   None  1513955069867274245  2022-04-12  NO_QUERY  OlafScholz   

                                                text  
0  Ich wünsche Ihnen und Ihren Familien frohe #Os...  
1  Meine Solidarität gilt @Karl_Lauterbach: Jeder...  
2  Sie haben Hilfe organisiert, Kräfte gebündelt ...  
3  Verantwortung übernehmen heißt: Deutschland li...  
4  Die Sanktionen gegen #Russland sind notwendig ...  
